# ETL

Utilizando o arquivo criado com o notebook download, vamos começar a processar os dados.

## Exploração inicial

In [ ]:
import pandas as pd

df = pd.read_csv('SIH_SUS_AL-2014-2023.csv')

In [13]:
df.head()

,UF_ZI,ANO_CMPT,MES_CMPT,CEP,MUNIC_RES,SEXO,IDADE,CID_NOTIF,VAL_TOT
0,270000,2014,1,57303785,270030,1.0,21.0,NaN,205.91
1,270000,2014,1,57260000,270420,1.0,28.0,NaN,426.29
2,270000,2014,1,57313060,270030,1.0,91.0,NaN,44.22
3,270000,2014,1,57560000,270250,1.0,28.0,NaN,40.38
4,270000,2014,1,57300620,270030,1.0,50.0,NaN,40.38


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1602663 entries, 0 to 1602662
Data columns (total 9 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   UF_ZI      1602663 non-null  int64  
 1   ANO_CMPT   1602663 non-null  int64  
 2   MES_CMPT   1602663 non-null  int64  
 3   CEP        1602663 non-null  int64  
 4   MUNIC_RES  1602663 non-null  int64  
 5   SEXO       1602662 non-null  float64
 6   IDADE      1602662 non-null  float64
 7   CID_NOTIF  1056 non-null     object 
 8   VAL_TOT    1602662 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 110.0+ MB


In [3]:
municipios_diferentes = df['MUNIC_RES'].nunique()
print(f'Existem {municipios_diferentes} códigos de municípios distintos.')

Existem 994 códigos de municípios distintos.


## Opcional

O código do IBGE do município de Maceió é 2704302.  

Notamos que o código de Maceió possuí 7 dígitos e os códigos no DataFrame possuem apenas 6 dígitos.

Inicialmente, podemos comparar os 6 primeiros dígitos do com os código do DataFrame e remover as demais cidades.

In [4]:
df1 = df[df['MUNIC_RES'] == 270430]
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 480626 entries, 228 to 1602307
Data columns (total 9 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   UF_ZI      480626 non-null  int64  
 1   ANO_CMPT   480626 non-null  int64  
 2   MES_CMPT   480626 non-null  int64  
 3   CEP        480626 non-null  int64  
 4   MUNIC_RES  480626 non-null  int64  
 5   SEXO       480626 non-null  float64
 6   IDADE      480626 non-null  float64
 7   CID_NOTIF  182 non-null     object 
 8   VAL_TOT    480626 non-null  float64
dtypes: float64(3), int64(5), object(1)
memory usage: 36.7+ MB


Dentro desses 480.626 linhas, podem existir municípios que não são Maceió.

Utilizando o CEP, podemos filtrar isso um pouco mais.

In [26]:
ceps_diferentes = df1['CEP'].nunique()
print(f'Existem {ceps_diferentes} CEPs distintos.')

CEPs = df1['CEP'].unique().tolist()

Existem 7115 CEPs distintos.


Poderiamos consultar esses CEPs em uma API qualquer, mas como os planos gratuítos são bem limitados, vamos ver se conseguimos diminuir esse número.  


Através da URL abaixo, podemos verificar que a faixa de CEPs da cidade de Maceió é a seguinte: 57000-001 a 57099-999  
https://buscacepinter.correios.com.br/app/faixa_cep_uf_localidade/index.php

Sabendo disso, podemos mapear as cidades de nosso DataFrame

In [44]:
import math

def checkMCZ(CEP):
    prefixo = math.floor(CEP/1000)
    if prefixo > 57000 and prefixo < 57099: return 'Maceió'
    return 'Outras'

df1['Cidade'] = df1['CEP'].apply(checkMCZ)

In [46]:
count_and_distinct = { 'CEP': ['count', 'nunique'] }

df1[['CEP', 'Cidade']].groupby(by='Cidade').agg(count_and_distinct)

CEP        
         count nunique
Cidade                
Maceió  480619    7114
Outras       7       1

O resultado acima nos leva a acreditar que existem 7114 CEPs pertencentes a Maceió e apenas 1 CEP de outra cidade.

In [47]:
somente_MCZ = df1['Cidade'] == 'Maceió'

df2 = df1[somente_MCZ]

df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 480619 entries, 228 to 1602307
Data columns (total 11 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   UF_ZI      480619 non-null  int64  
 1   ANO_CMPT   480619 non-null  int64  
 2   MES_CMPT   480619 non-null  int64  
 3   CEP        480619 non-null  int64  
 4   MUNIC_RES  480619 non-null  int64  
 5   SEXO       480619 non-null  float64
 6   IDADE      480619 non-null  float64
 7   CID_NOTIF  182 non-null     object 
 8   VAL_TOT    480619 non-null  float64
 9   MCZ        480619 non-null  object 
 10  Cidade     480619 non-null  object 
dtypes: float64(3), int64(5), object(3)
memory usage: 44.0+ MB


Observação: Eu notei que eu poderia ter simplesmente pulado a filtragem através MUNIC_RES, portanto estou marcando essa parte como opcional.

## Filtrando apenas Maceió

Faixa de CEPs da cidade de Maceió é a seguinte: 
- 57000-001 a 57099-999 ¹

Sabendo disso, podemos filtrar apenas Maceió através da coluna de CEP.  

<sup><sub>¹ Dados obtidos em https://buscacepinter.correios.com.br/app/faixa_cep_uf_localidade/index.php</sub></sup>

In [48]:
import math

def checkMCZ(CEP):
    prefixo = math.floor(CEP/1000)
    if prefixo > 57000 and prefixo < 57099: return 'Maceió'
    return 'Outras'

df['Cidade'] = df['CEP'].apply(checkMCZ)

somente_MCZ = df['Cidade'] == 'Maceió'

df1 = df[somente_MCZ]

df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 480619 entries, 228 to 1602307
Data columns (total 10 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   UF_ZI      480619 non-null  int64  
 1   ANO_CMPT   480619 non-null  int64  
 2   MES_CMPT   480619 non-null  int64  
 3   CEP        480619 non-null  int64  
 4   MUNIC_RES  480619 non-null  int64  
 5   SEXO       480619 non-null  float64
 6   IDADE      480619 non-null  float64
 7   CID_NOTIF  182 non-null     object 
 8   VAL_TOT    480619 non-null  float64
 9   Cidade     480619 non-null  object 
dtypes: float64(3), int64(5), object(2)
memory usage: 40.3+ MB


In [51]:
df1.groupby(by='CID_NOTIF').count()

,UF_ZI,ANO_CMPT,MES_CMPT,CEP,MUNIC_RES,SEXO,IDADE,VAL_TOT,Cidade
CID_NOTIF,,,,,,,,,
O101,3,3,3,3,3,3,3,3,3
O102,1,1,1,1,1,1,1,1,1
O140,1,1,1,1,1,1,1,1,1
O993,1,1,1,1,1,1,1,1,1
Z302,176,176,176,176,176,176,176,176,176
